In [1]:
import numpy as np
import pandas as pd
import math as m
import sympy as sym
import random as rand
import matplotlib.pyplot as plt
import cmath as cm
# Testing..attention please..nuisance, you sent, who sent for me?

In [2]:
# defininf the states
# defininsg coefficeients sybols

def Creating_states( coeff = [1/(2**.5),1/(2**.5),1/(2**.5),1/(2**.5)] , Abstract = False):    # coeff list like [a0,a1,b0,b1]
    if Abstract == True:
        a0 = sym.symbols('a0')
        a1 = sym.symbols('a1')
        b0 = sym.symbols('b0')
        b1 = sym.symbols('b1')
    else:
        a0 = coeff[0]
        a1 = coeff[1]
        b0 = coeff[2]
        b1 = coeff[3]
    psi0 = [a0,a1]      # defining states
    psi1 = [b0,b1]
    return([psi0,psi1])

In [3]:
psi0 = Creating_states(Abstract=False)[0]
psi1 = Creating_states(Abstract=False)[1]

psi0

[0.7071067811865475, 0.7071067811865475]

In [4]:
psi0sq = []
psi1sq = []
[[psi0sq.append(i*j) for i in psi0] for j in psi0]
[[psi1sq.append(i*j) for i in psi1] for j in psi1]   # creating list of coeff for squared states (intermeditate step)

psi0psi0 = [psi0sq[0], (psi0sq[1]**2+psi0sq[2]**2)**(.5), psi0sq[3]]        # the second element is the normalized coefficient due to the addition of states (01+10). See OneNote writeup for details
psi1psi1 = [psi1sq[0], (psi1sq[1]**(2)+psi1sq[2]**(2))**(.5), psi1sq[3]]            # symmetric states in basis (|00>, normalized version of (|01>+|10>),|11>)

# psi0psi0
# sum([i**2 for i in psi0psi0])   # adding to 1 now.

In [5]:
#creating the density matrix rho
# rho is 1/2*(|psi0psi0Xpsi0psi0|+|psi1psi1Xpsi1psi1|)
# findiing the conjugate vectors 
vec_psi0psi0 = sym.Matrix(psi0psi0)     # convertying to sympy vectors for easy matrix multiplication
vec_psi1psi1 = sym.Matrix(psi1psi1)

rho = (1/2)*(vec_psi0psi0*vec_psi0psi0.T  + vec_psi1psi1*vec_psi1psi1.T)       # density matrix with priors 1/2 each.
rho

Matrix([
[             0.25, 0.353553390593274,              0.25],
[0.353553390593274,               0.5, 0.353553390593274],
[             0.25, 0.353553390593274,              0.25]])

In [6]:
# creating the SIC POVM matrices
w = m.e**((2/3)*m.pi*(1j))
POVM_vec = (1/(2**.5))*(np.array([[0,1,-1],[-1,0,1],[1,-1,0],[0,w,-w**2],[-1,0,w**2],[1,-w,0],[0,w**2,-w],[-1,0,w],[1,-w**2,0]]))
POVM_elts = [(1/3)*np.outer(np.conjugate(POVM_vec[i]),POVM_vec[i]) for i in range(len(POVM_vec))]
# sum([np.trace(POVM_elts[i]) for i in range(9)])`# trace sum checks out`

In [7]:
# prob_vec = [(POVM_elts[i]*rho).trace() for i in range(len(POVM_elts))]
prob_vec =  [np.trace(np.dot(POVM_elts[i],rho)) for i in range(9)] #[(POVM_elts[i]*rho).trace() for i in range(len(POVM_elts))]
# prob_vec
sum(prob_vec)        # prob not adding to 1?
# print(prob_vec)


0.999999999999999

In [8]:
# # verifying that the trace still holds to be the same, equalt to .111 for this weird POVM element.
# c = .00308641975308642*2.71828182845905**(4.18879020478639*(sym.I))

# p = cm.polar(c)
# cm.rect(p[0], p[1])

# r = .00617-.0015
# i = .0026

# ab = (r**2+i**2)**(.5)
# 2*ab # verified

In [87]:
N = 10000       # number of measurements  

POVM_dir_symbols = ['d1','d2','d3','d4','d5','d6','d7','d8','d9']       # symbols to indicate collapsed direction
#prob distribution is simply the corresponding elements of the prob_vec
collapse_dir_vec = rand.choices(POVM_dir_symbols, weights=prob_vec, k = N)   # choosing collapse directions with weights for N trials

nj_vec = [collapse_dir_vec.count(f'd{i+1}') for i in range(9)]         # working babieee
pj_num_vec = [i/N for i in nj_vec]         # working babieee
([i*N for i in prob_vec] , nj_vec)   # comparing the theoretical and experimental collapse prob.

([71.4886980224206,
  0,
  71.4886980224206,
  1839.25565098879,
  1250.00000000000,
  1839.25565098879,
  1839.25565098879,
  1250.00000000000,
  1839.25565098879],
 [62, 0, 66, 1789, 1298, 1800, 1926, 1213, 1846])

In [64]:
M = [[np.trace(np.dot(POVM_elts[i],POVM_elts[j])) for i in range(len(POVM_elts))] for j in range(len(POVM_elts))]     # creating M matrix using POVM definition


In [90]:
u_0 = [1/3 for i in range(9)]           # cerating u_0 vector, to create the inverse matrix
M_inv = 3*np.outer(u_0,u_0) + 12*(np.eye(9) - np.outer(u_0,u_0))        # creating the inverse matrix
# np.trace(np.dot(M,M_inv))       # gives identity, checks out
r_vec = np.dot(M_inv,pj_num_vec)


array([-0.9256, -1.    , -0.9208,  1.1468,  0.5576,  1.16  ,  1.3112,
        0.4556,  1.2152])

In [93]:
rho_num_list = [r_vec[i]*POVM_elts[i] for i in range(len(POVM_elts))]

[i for i in rho_num_list]


[array([[-0.        +0.j, -0.        +0.j, -0.        +0.j],
        [-0.        +0.j, -0.15426667+0.j,  0.15426667-0.j],
        [-0.        +0.j,  0.15426667+0.j, -0.15426667+0.j]]),
 array([[-0.16666667+0.j, -0.        +0.j,  0.16666667+0.j],
        [-0.        +0.j, -0.        +0.j, -0.        +0.j],
        [ 0.16666667-0.j, -0.        +0.j, -0.16666667+0.j]]),
 array([[-0.15346667+0.j,  0.15346667-0.j, -0.        +0.j],
        [ 0.15346667+0.j, -0.15346667+0.j, -0.        +0.j],
        [-0.        +0.j, -0.        +0.j, -0.        +0.j]]),
 array([[0.        +0.00000000e+00j, 0.        +0.00000000e+00j,
         0.        +0.00000000e+00j],
        [0.        +0.00000000e+00j, 0.19113333+4.02832527e-18j,
         0.09556667-1.65526322e-01j],
        [0.        +0.00000000e+00j, 0.09556667+1.65526322e-01j,
         0.19113333-8.95212367e-19j]]),
 array([[ 0.09293333+0.00000000e+00j,  0.        +0.00000000e+00j,
          0.04646667+8.04826275e-02j],
        [ 0.        +0.00000

In [97]:
rho_num = np.zeros_like(rho_num_list[0])

# Loop over the matrices and sum them element-wise
for matrix in rho_num_list:
    rho_num = np.add(rho_num, matrix)

rho_num 

array([[0.2446+0.00000000e+00j, 0.3514+7.96743371e-03j,
        0.2511+1.47224319e-02j],
       [0.3514-7.96743371e-03j, 0.4978+6.13086519e-18j,
        0.3591+2.37290961e-02j],
       [0.2511-1.47224319e-02j, 0.3591-2.37290961e-02j,
        0.2576+4.87569334e-18j]])

In [98]:
rho

Matrix([
[             0.25, 0.353553390593274,              0.25],
[0.353553390593274,               0.5, 0.353553390593274],
[             0.25, 0.353553390593274,              0.25]])